In [9]:
from utils import *
import time
from sklearn.svm import LinearSVC, SVC
from copy import deepcopy
import numpy as np
import scipy.io as sio

import math
import tensorflow as tf
from scipy import signal
from sklearn.model_selection import train_test_split

from sklearn.feature_selection import mutual_info_classif

'''
from mutual_information import mutual_information 
        https://gist.github.com/elsonidoq/4230222
        doesn't make any deffrence in output    
'''
import math
import time
import pyriemann.utils.mean as rie_mean
from scipy.signal import hilbert
from scipy import linalg

import pandas as pd

#get_data.py loads the dataset for a spceific subject, and the option of whether the training data or testing data
#utils.py have the underlying functions such as extract features function and the filter bank that allow for the FBCSP class to work
#the FBCSP class showcases the different function that work hand in hand to provied the FBCSP algorithm

#Figure out how to utilize the FBCSP Class
#Include code from Utils.py and COMMENTS
#Test classifier
#Retrieve Random Data Pointt

In [10]:
#GETDATA FUNCTION

def get_data(subject,training,PATH):
    '''
    Loads the dataset 2a of the BCI Competition IV
    available on http://bnci-horizon-2020.eu/database/data-sets

    Keyword arguments:
    subject -- number of subject in [1, .. ,9]
    training -- if True, load training data
                if False, load testing data

    Return: data_return 	numpy matrix 	size = NO_valid_trial x 22 x 1750
    class_return 	numpy matrix 	size = NO_valid_trial
    '''
    NO_channels = 22
    NO_tests = 6*48
    Window_Length = 7*250

    class_return = np.zeros(NO_tests)
    data_return = np.zeros((NO_tests,NO_channels,Window_Length))

    NO_valid_trial = 0
    print("PATH : ",PATH,"  subject : ",subject)
    if training:
        a = sio.loadmat(PATH+'test.mat')
    else:
        a = sio.loadmat(PATH+'test.mat')
    a_data = a['TrData']
    
    for ii in range(0,a_data.size):
        a_data1 = a_data[0,ii]
        a_data2=[a_data1[0,0]]
        a_data3=a_data2[0]
        a_X= a_data3[0]
        a_trial= a_data3[1]
        a_y = a_data3[2]
        a_fs= a_data3[3]
        a_classes= a_data3[4]
        a_artifacts= a_data3[5]
        a_gender= a_data3[6]
        a_age= a_data3[7]
        for trial in range(0,a_trial.size):
            if(a_artifacts[trial]==0):
                data_return[NO_valid_trial,:,:] = np.transpose(a_X[int(a_trial[trial]):(int(a_trial[trial])+Window_Length),:22])
                class_return[NO_valid_trial] = int(a_y[trial])
                NO_valid_trial +=1


    return data_return[0:NO_valid_trial,:,:], class_return[0:NO_valid_trial]

In [13]:
from get_data import get_data

class FBCSP_Model:
    def __init__(self ,
                 Path = 'C:/Users/DLSU/Desktop/Multiple Classes FBCSP/datasets/',
                 Fs = 250,
                 Svm_kernel = 'linear',
                 Svm_c = 0.15,
                 NO_channels = 22,
                 NO_subjects = 9,
                 NO_weights = 3,
                 NO_class = 4,
                 selection = True,
                 NO_selection = 8,
                 class_selection_base = True,
                 Output = 'csp-space',
                 Bands = [[4,8],[8,12],[12,16],[16,20],[20,24],[24,28]],
                 OneVersuseOne = False,
                ):
        '''
        filterbank csp model
        arguments :
            Path                 -- path to where data set is saved 'String'
            Fs                   -- sampling frequency 'Integer'
            Svm_kernel           -- svm kernel for classification 'String'
            Svm_c                -- svm cost parameter 'Float'
            NO_channels          -- number of channel 'Integer'
            NO_subjects          -- number of subject in dataset 'Integer'
            NO_weights           -- parameter M in csp algorithm 'Integer'
            NO_class             -- number of classess 'Integer'
            selection            -- whether or not feature selection will perform  'Boolean'
            NO_selection         -- number of feature to be selected after transforming into csp-space 'Integer'
            class_selection_base -- whether or not feature selection perform with regarding class 'Boolean'
            Output               -- what will be output of run_csp function / 'power': accuracy  'csp-space' : transformed signal
            Bands                -- Filter Bank bandwidth
            OneVersuseOne        -- whether or not perform one versuse one or one versuse all 'Boolean'
            
        '''
        self.data_path = Path
        self.svm_kernel = Svm_kernel #'sigmoid'#'linear' # 'sigmoid', 'rbf', 'poly'
        self.svm_c = Svm_c # 0.05 for linear, 20 for rbf
        self.fs = Fs # sampling frequency 
        self.NO_channels = NO_channels # number of EEG channels 
        self.NO_subjects = NO_subjects
        self.NO_weights  = NO_weights # Total number of CSP feature per band
        self.NO_selection = NO_selection # number of feature to be selected 
        self.NO_class    = NO_class
        self.class_selection_base = class_selection_base
        self.bw = deepcopy(Bands) # bandwidth of filtered signals 
        self.Output = Output
        self.OnevsOne = OneVersuseOne 
        self.selection = selection
        self.f_bands_nom = load_bands(self.bw,self.fs) # get normalized bands 

        self.NO_bands = len(self.f_bands_nom)
        #self.NO_time_windows = int(self.time_windows.size/2)
        if(self.OnevsOne):
            self.NO_features = 2 * self.NO_weights * self.NO_bands * ((self.NO_class * (self.NO_class-1))/2)
        else :
            self.NO_features = 2 * self.NO_weights * self.NO_bands * self.NO_class
        
        self.train_time = 0
        self.train_trials = 0
        self.eval_time = 0
        self.eval_trials = 0

    def run_csp(self):
        '''
        start to train filter bank csp model
        '''
        ########### Training ############
        start_train = time.time()
        #  Apply CSP to bands to get spatial filter 
        w = generate_projection(self.train_data,
                                self.train_label,
                                self.f_bands_nom,
                                self.NO_weights,
                                self.NO_class,
                                self.OnevsOne
                               )
        #  Extract feature after transforming signal with calculated filter in variable "w"
        feature_mat = extract_feature(self.train_data,w,self.f_bands_nom)

        #  check whether feature selection should be performed or not
        if(self.selection):
            if(self.class_selection_base):
                self.selected_features = select_feature_class(feature_mat,
                                                  self.train_label,
                                                  self.NO_weights,
                                                  self.NO_selection,
                                                  self.NO_class)
                
                print("Number of selected feature ",sum(len(x) for x in self.selected_features))
            else:
                self.selected_features = select_feature_all(feature_mat,
                                      self.train_label,
                                      self.NO_weights,
                                      self.NO_selection,
                                      self.NO_class)
                
                print("Number of selected feature ",len(self.selected_features))
            
        if(self.Output == 'power'):
            print("")
            print("extracting feature and classification")
            if(self.selection):
                # select selected feature if feature selection is performed earlier
                if(self.class_selection_base):
                    feature_mat = reduce_feature_class(feature_mat,
                                                 self.selected_features,
                                                 self.NO_weights)
                else:
                    feature_mat = reduce_feature_all(feature_mat,
                                                 self.selected_features,
                                                 self.NO_weights)
            
            else :
                # flattening signal if no feature selection is performed 
                feature_mat = feature_mat.reshape((feature_mat.shape[0],-1))
            
            
            # 3. Stage Train SVM Model 
            if self.svm_kernel == 'linear' : 
                clf = LinearSVC(C = self.svm_c,
                                intercept_scaling=1,
                                loss='hinge',
                                max_iter=1000,
                                multi_class='ovr',
                                penalty='l2',
                                random_state=1,
                                tol=0.00001)
            else:
                clf = SVC(self.svm_c,
                          self.svm_kernel, 
                          degree=10, 
                          gamma='auto',
                          coef0=0.0,
                          tol=0.001, 
                          cache_size=10000, 
                          max_iter=-1, 
                          decision_function_shape='ovr')
            
            clf.fit(feature_mat,self.train_label) 
            print(clf.predict((feature_mat)))
            # print(feature_mat[0,:])
            # print(feature_mat.shape)
            # print(clf.predict(feature_mat[0,:]))
            end_train = time.time()
            self.train_time += end_train-start_train
            self.train_trials += len(self.train_label)

            ################################# Evaluation ###################################################
            start_eval = time.time()
            eval_feature_mat = extract_feature(self.eval_data,w,self.f_bands_nom)
            if(self.selection):
                if(self.class_selection_base):
                    eval_feature_mat = reduce_feature_class(eval_feature_mat,
                                                      self.selected_features,
                                                      self.NO_weights)
                else :
                    eval_feature_mat = reduce_feature_all(eval_feature_mat,
                                                      self.selected_features,
                                                      self.NO_weights)
            else :
                eval_feature_mat = eval_feature_mat.reshape((eval_feature_mat.shape[0],-1))
            
            success_rate     = clf.score(eval_feature_mat,self.eval_label)

            end_eval = time.time()

            print("Time for one Evaluation " + str((end_eval-start_eval)/len(self.eval_label)) )

            self.eval_time += end_eval-start_eval
            self.eval_trials += len(self.eval_label)
            
            return success_rate
        
        else :
            # set output to be csp-space if "self.Output" is not power
            if(self.selection):
                # if selection is performed earlier
                # how feature selection is performed ? with regards or concatenation
                if(self.class_selection_base):
                    train_transformed = transform_class(self.train_data,
                                                  w,
                                                  self.f_bands_nom,
                                                  self.selected_features,
                                                  self.NO_selection,
                                                  self.NO_weights)

                    eval_transformed = transform_class(self.eval_data,
                                                 w,
                                                 self.f_bands_nom,
                                                 self.selected_features,
                                                 self.NO_selection,
                                                 self.NO_weights)
                else :
                    train_transformed = transform_all(self.train_data,
                                                  w,
                                                  self.f_bands_nom,
                                                  self.selected_features,
                                                  self.NO_selection,
                                                  self.NO_weights)

                    eval_transformed = transform_all(self.eval_data,
                                                 w,
                                                 self.f_bands_nom,
                                                 self.selected_features,
                                                 self.NO_selection,
                                                 self.NO_weights)
            else :
                # if no feature selection is performed
                train_transformed = transform(self.train_data,
                                              w,
                                              self.f_bands_nom)
                
                eval_transformed = transform(self.eval_data,
                                             w,
                                             self.f_bands_nom)
                
            return [train_transformed,self.train_label,eval_transformed,self.eval_label]


    def load_data(self):
        '''
        load data from "self.path" variable
        '''
        self.train_data,self.train_label = get_data(self.subject,True,self.data_path)
        #self.train_data  = self.train_data[0:20]
        #self.train_label = self.train_label[0:20]
        self.eval_data,self.eval_label = get_data(self.subject,False,self.data_path)
        

SyntaxError: invalid syntax (4247496240.py, line 148)

In [12]:
def gevd(x1,x2,no_pairs):
    '''
    Solve generalized eigenvalue decomposition
    Keyword arguments:
        x1 -- numpy array of size [NO_channels, NO_samples]
        x2 -- numpy array of size [NO_channels, NO_samples]
        no_pairs -- number of pairs of eigenvectors to be returned 
    Return:
        numpy array of 2*No_pairs eigenvectors 
    '''
    ev,vr= linalg.eig(x1,x2,right=True) 
    evAbs = np.abs(ev)
    sort_indices = np.argsort(evAbs)
    chosen_indices = np.zeros(2*no_pairs).astype(int)
    chosen_indices[0:no_pairs] = sort_indices[0:no_pairs]
    chosen_indices[no_pairs:2*no_pairs] = sort_indices[-no_pairs:]
    
    w = vr[:,chosen_indices] # ignore nan entries 
    return w

def csp_one_one(cov_matrix,NO_csp ,NO_pairs):
    '''
    calculate spatial filter for class (1,2) (1,3) (1,4) (2,3) (2,4) (3,4)
    Keyword arguments:
    cov_matrix -- numpy array of size [N_classes , NO_channels, NO_channels]
    NO_csp -- number of spatial filters
    Return:	spatial filter numpy array of size [NO_channels, NO_csp] 
    '''
    N, _ = cov_matrix[0].shape # N is number of channel
    w = np.zeros((N,NO_csp))
    kk = 0 # internal counter
    for cc1 in range(0,4):
        for cc2 in range(cc1+1,4):
            #generalized eigen value decompositon between classess
            w[:,NO_pairs*2*(kk):NO_pairs*2*(kk+1)] = gevd(cov_matrix[cc1], cov_matrix[cc2],NO_pairs)
            kk +=1
    return w

def csp_one_all(cov_matrix,NO_csp ,NO_pairs):
    '''
    calculate spatial filter for class (1 vs other ) (2 vs other ) (3 vs other ) (4 vs other ) 
    Keyword arguments:
    cov_matrix -- numpy array of size [N_classes ,NO_channels, NO_channels]
    NO_csp -- number of spatial filters
    Return:	spatial filter numpy array of size [ NO_channels ,NO_csp] 
    '''
    N_class , N, _ = cov_matrix.shape # N is number of channel
    
    w = np.zeros((N,NO_csp))
    kk = 0 # internal counter
    for classCov in range(0,4):
        #covariance average of other
        covAvg = rie_mean.mean_covariance(cov_matrix[ np.arange(0,N_class) != classCov,:,:], metric = 'euclid')
        w[:,NO_pairs*2*(kk):NO_pairs*2*(kk+1)] = gevd(cov_matrix[classCov], covAvg ,NO_pairs)
        kk +=1
    return w

def generate_projection(data,
                        class_vec,
                        f_bands_nom,
                        NO_weights,
                        NO_class,
                        OnevsOne): 
    '''
    generate spatial filters for every frequancy band and return weight matrix
    
    Keyword arguments:
    data         -- numpy array of size [NO_trials,channels,time_samples]
    class_vec    -- containing the class labels, numpy array of size [NO_trials]
    NO_weights   -- number of weights ,
    NO_class     -- number of classes,
    f_bands_nom  -- numpy array [[start_freq1,end_freq1],...,[start_freqN,end_freqN]]
    time_windows -- numpy array [[start_time1,end_time1],...,[start_timeN,end_timeN]] 

    Return: spatial filter numpy array of size [NO_timewindows,NO_freqbands,22,NO_csp] 
    '''
    NO_bands = len(f_bands_nom)
    NO_channels = len(data[0,:,0])
    NO_trials = class_vec.size
    if(OnevsOne):
        NO_csp = np.int(((NO_class * (NO_class-1))/2) * NO_weights * 2)
    else :
        NO_csp = np.int(NO_class * NO_weights * 2)
    # Initialize spatial filter: 
    w = np.zeros((NO_bands,NO_channels,NO_csp))
    # iterate through all time windows 
    
    # get start and end point of current time window 
    # *must get value in arguments
    t_start = int(2.5 * 250)
    t_end = int(6 * 250)

    # iterate through all frequency bandwids 
    print("Calculate filter for band : ")
    for subband in range(0,NO_bands): 
        print(subband,",", end=" ")
        cov      = np.zeros((4, NO_trials, NO_channels, NO_channels)) # sum of covariance depending on the class
        cov_avg  = np.zeros((4, NO_channels, NO_channels))
        cov_cntr = np.zeros(4).astype(int) # counter of class occurence 

        #go thrugh all trials and estimate covariance matrix of every class 
        for trial in range(0,NO_trials):
            #frequency band of every channel
            data_filter = bandpass_filter(data[trial,:,t_start:t_end],f_bands_nom[subband])
            # must calculae indecies of class label for automation
            cur_class_idx = int(class_vec[trial]-1)

            # caclulate current covariance matrix 
            cov[cur_class_idx,cov_cntr[cur_class_idx],:,:] = np.dot(data_filter,np.transpose(data_filter))

            # update covariance matrix and class counter 
            cov_cntr[cur_class_idx] += 1

        # calculate average of covariance matrix 
        for clas in range(0,4):
            cov_avg[clas,:,:] = rie_mean.mean_covariance(cov[clas,:cov_cntr[clas],:,:], metric = 'euclid')
        if(OnevsOne):
            w[subband,:,:] = csp_one_one(cov_avg, NO_csp, NO_weights)
        else :
            w[subband,:,:] = csp_one_all(cov_avg, NO_csp, NO_weights)
    return w


def extract_feature(data,w,f_bands_nom):
    '''
    calculate log variance features using the precalculated spatial filters
    
    Keyword arguments:
    data         -- numpy array of size [NO_trials,channels,time_samples]
    w            -- spatial filters, numpy array of size [NO_timewindows,NO_freqbands,22,NO_csp]
    f_bands_nom  -- numpy array [[start_freq1,end_freq1],...,[start_freqN,end_freqN]]
    time_windows -- numpy array [[start_time1,end_time1],...,[start_timeN,end_timeN]] 

    Return: features, numpy array of size [NO_trials,(NO_csp*NO_bands*NO_time_windows)]
    
    '''
    NO_csp = len(w[0,0,:])
    NO_bands = len(f_bands_nom)
    NO_trials = len(data[:,0,0])
    NO_features = NO_csp * NO_bands
    feature_mat = np.zeros(( NO_trials, NO_bands, NO_csp))
    
    # initialize feature vector
    feat = np.zeros((NO_bands,NO_csp))

    # go through all trials 
    t_start = int(2.5 * 250)
    t_end = int(6 * 250)
    
    for trial in range(0,NO_trials):
        for subband in range(0,NO_bands):
            #Apply spatial Filter to data 
            cur_data_s = np.dot(np.transpose(w[subband]),data[trial,:,t_start:t_end])

            #frequency filtering  
            cur_data_f_s = bandpass_filter(cur_data_s,f_bands_nom[subband])

            # calculate variance of all channels 
            feat[subband] = np.var(cur_data_f_s,axis=1)
        # calculate log10 of normalized feature vector 

        for subband in range(0,NO_bands):
            feat[subband] = np.log10(feat[subband]/np.sum(feat[subband]))
            #feat[subband] = (feat[subband]/np.sum(feat[subband]))

         # store feature in list
        feature_mat[trial,:,:] = feat
        
    return feature_mat
    #return np.reshape(feature_mat,(NO_trials,-1))

def select_feature_class(feature_mat,label,N_pair,N_selection,N_class):
    '''
    find best index with mutual information based feature selection regarding each class
    without concatenation
    
    Keyword arguments:
    feature_mat   -- numpy array of size [NO_trials,Bands,classes * 2 * pair]
    N_selection   -- number of channel to select
    N_pair        -- number of pair
    N_class       -- number of classes
    
    Return: selected feature for each class , array of size [Number of Classes,(Selected index)] 
    '''
    channels = feature_mat.shape[1] * feature_mat.shape[2]
    feature_mat =  np.swapaxes(feature_mat,1,2)
    
    bins = np.arange(0,channels+1,2*N_pair)

    class_selected = [[] for _ in range(N_class)]
    trans = {}

    for i , j in zip(range(2*N_pair) , reversed(range(2*N_pair))):
        trans[i] = j
        
    for i_class in range(N_class):
        data = feature_mat[:,(i_class * 2 * N_pair) : ((i_class + 1) * 2 * N_pair),:]
        data = np.reshape(data,(data.shape[0],-1))
        c_label = label == i_class+1
        Mi = mutual_info_classif(data, c_label.ravel() ,discrete_features = False,n_neighbors = 50)
        selected = np.argsort(-Mi)
        binSelected = np.digitize(selected,bins)
        finalSelected = set()
        counter = 0
        i_selected = 0
        while(counter < N_selection and i_selected < binSelected.shape[0] ):
            if(selected[i_selected] not in finalSelected):
                finalSelected.add(selected[i_selected])
                pair = trans[selected[i_selected] - bins[binSelected[i_selected]-1]] + bins[binSelected[i_selected]-1]
                finalSelected.add(pair)
                counter += 1
            i_selected +=1

        class_selected[i_class] = list(finalSelected)
    return class_selected

def select_feature_all(feature_mat,label,N_pair,N_selection,N_class):
    '''
    find best index with mutual information based feature selection after concatenating all of
    the feature extracted from bands
    
    Keyword arguments:
    feature_mat  -- numpy array of size [NO_trials,Bands,classes * 2 * pair]
    N_selection  -- number of channel to select
    N_pair       -- number of pair
    N_class      -- number of classes
    
    Return: selected feature for each class,array of size [Number of Classes,(Selected index)] 
    
    '''
    channels = feature_mat.shape[1] * feature_mat.shape[2]
    feature_mat =  np.swapaxes(feature_mat,1,2)
    
    bins = np.arange(0,channels+1,2*N_pair)

    trans = {}
    for i , j in zip(range(2*N_pair) , reversed(range(2*N_pair))):
        trans[i] = j
        
    #data = feature_mat[:,(i_class * 2 * N_pair) : ((i_class + 1) * 2 * N_pair),:]
    
    data = np.reshape(feature_mat,(feature_mat.shape[0],-1))
    Mi = mutual_info_classif(data,label.ravel() ,discrete_features = False,n_neighbors = 50)
    
    selected = np.argsort(-Mi)
    print(selected)
    binSelected = np.digitize(selected,bins)
    finalSelected = set()
    
    counter = 0
    i_selected = 0
    
    while(counter < N_selection and i_selected < binSelected.shape[0] ):
        if(selected[i_selected] not in finalSelected):
            finalSelected.add(selected[i_selected])
            pair = trans[selected[i_selected] - bins[binSelected[i_selected]-1]] + bins[binSelected[i_selected]-1]
            finalSelected.add(pair)
            counter += 1
        i_selected +=1

    return list(finalSelected)

def reduce_feature_class(feature_mat,class_selected,N_pair):
    '''
    generate new data set based on feature selected regarding class
    
    Keyword arguments:
    feature_mat     -- numpy array of size [NO_trials,Bands,classes * 2 * pair]
    class_selected  -- list of size [ N_class , Selected Index]
    N_pair          -- number of pair
    
    Return: new data
    ''' 
    feature_mat =  np.swapaxes(feature_mat,1,2)
    new_feature_mat = np.zeros((feature_mat.shape[0],0))
    for i_class in range(len(class_selected)):
        data = feature_mat[:,(i_class * 2 * N_pair) : ((i_class + 1) * 2 * N_pair),:]
        data = np.reshape(data,(data.shape[0],-1))
        new_feature_mat = np.hstack((new_feature_mat,data[:,class_selected[i_class]]))
    return new_feature_mat

def reduce_feature_all(feature_mat,selected,N_pair):
    '''
    generate new data set based on feature selected after concatenation
    
    Keyword arguments:
    feature_mat     -- numpy array of size [NO_trials,Bands,classes * 2 * pair]
    selected        -- list of size [ Selected Index]
    N_pair          -- number of pair
    
    Return: new data
    ''' 
    feature_mat = np.swapaxes(feature_mat,1,2)
    feature_mat = np.reshape(feature_mat,(feature_mat.shape[0],-1))
    feature_mat = feature_mat[:,selected]
    return feature_mat

def transform(data,w,f_bands_nom):
    '''
    tramsform input data into csp space when no feature selection is performed
    
    Keyword arguments:
    data         -- numpy array of size [NO_trials,channels,time_samples]
    w            -- spatial filters, numpy array of size [NO_timewindows,NO_freqbands,22,NO_csp]
    f_bands_nom  -- numpy array [[start_freq1,end_freq1],...,[start_freqN,end_freqN]]

    Return: features, numpy array of size [NO_trials,(NO_csp*NO_bands)] 
    '''
    t_start = int(2.5 * 250)
    t_end = int(6 * 250)
    NO_csp = len(w[0,0,:])
    NO_bands = len(f_bands_nom)
    NO_trials = len(data[:,0,0])
    NO_features = NO_csp*NO_bands
    feature_mat = np.zeros(( NO_trials, NO_bands, NO_csp,t_end - t_start))
    
    # initialize feature vector
    feat = np.zeros(( NO_bands, NO_csp, t_end-t_start))

    # go through all trials 
    t_start = int(2.5 * 250)
    t_end = int(6 * 250)
    
    for trial in range(0,NO_trials):
        for subband in range(0,NO_bands):
            #Apply spatial Filter to data 
            cur_data_s = np.dot(np.transpose(w[subband]),data[trial,:,t_start:t_end])

            #frequency filtering  
            feat[subband] = bandpass_filter(cur_data_s,f_bands_nom[subband])

         # store feature in list
        feature_mat[trial,:,:,:] = feat
    feature_mat = np.reshape(feature_mat,(feature_mat.shape[0],-1,feature_mat.shape[-1]))
    return feature_mat
        
def transform_class(data,w,f_bands_nom,class_selected,N_selection,N_pair):
    '''
    tramsform input data into csp space when feature selection is performed regarding each class
    
    Keyword arguments:
    data         -- numpy array of size [NO_trials,channels,time_samples]
    w            -- spatial filters, numpy array of size [NO_timewindows,NO_freqbands,22,NO_csp]
    f_bands_nom  -- numpy array [[start_freq1,end_freq1],...,[start_freqN,end_freqN]]

    Return: features, numpy array of size [NO_trials,(sum of selected index *NO_bands)] 
    '''
    t_start = int(2.5 * 250)
    t_end = int(6 * 250)
    NO_csp = len(w[0,0,:])
    NO_bands = len(f_bands_nom)
    NO_trials = len(data[:,0,0])
    NO_features = NO_csp*NO_bands
    feature_mat = np.zeros(( NO_trials, NO_bands, NO_csp,t_end - t_start))
    
    # initialize feature vector
    feat = np.zeros(( NO_bands, NO_csp, t_end-t_start))

    # go through all trials 
    t_start = int(2.5 * 250)
    t_end = int(6 * 250)
    
    for trial in range(0,NO_trials):
        for subband in range(0,NO_bands):
            #Apply spatial Filter to data 
            cur_data_s = np.dot(np.transpose(w[subband]),data[trial,:,t_start:t_end])

            #frequency filtering  
            feat[subband] = bandpass_filter(cur_data_s,f_bands_nom[subband])

         # store feature in list
        feature_mat[trial,:,:,:] = feat
    
    N_class = len(class_selected)
    feature_mat =  np.swapaxes(feature_mat,1,2)
    new_feature_mat = np.zeros((feature_mat.shape[0],N_class * 2 * N_selection,t_end - t_start))
    
    for i_class in range(N_class):
        data = feature_mat[:,(i_class * 2 * N_pair) : ((i_class + 1) * 2 * N_pair),:,:]
        data = np.reshape(data,(data.shape[0],data.shape[1] * data.shape[2] , data.shape[3]))
        new_feature_mat[:,((i_class)*2 * N_selection) : ((i_class + 1)*2 * N_selection) ,:] = data[:,class_selected[i_class],:]

    return new_feature_mat

def transform_all(data,w,f_bands_nom,selected,N_selection,N_pair):
    '''
    tramsform input data into csp space when feature selection is performed after concatenation of feature
    
    Keyword arguments:
    data         -- numpy array of size [NO_trials,channels,time_samples]
    w            -- spatial filters, numpy array of size [NO_timewindows,NO_freqbands,22,NO_csp]
    f_bands_nom  -- numpy array [[start_freq1,end_freq1],...,[start_freqN,end_freqN]]

    Return: features, numpy array of size [NO_trials,(sum of selected index *NO_bands)] 
    '''
    t_start = int(2.5 * 250)
    t_end = int(6 * 250)
    NO_csp = len(w[0,0,:])
    NO_bands = len(f_bands_nom)
    NO_trials = len(data[:,0,0])
    NO_features = NO_csp*NO_bands
    feature_mat = np.zeros(( NO_trials, NO_bands, NO_csp,t_end - t_start))
    
    # initialize feature vector
    feat = np.zeros(( NO_bands, NO_csp, t_end-t_start))

    # go through all trials 
    t_start = int(2.5 * 250)
    t_end = int(6 * 250)
    
    for trial in range(0,NO_trials):
        for subband in range(0,NO_bands):
            #Apply spatial Filter to data 
            cur_data_s = np.dot(np.transpose(w[subband]),data[trial,:,t_start:t_end])

            #frequency filtering  
            feat[subband] = bandpass_filter(cur_data_s,f_bands_nom[subband])

         # store feature in list
        feature_mat[trial,:,:,:] = feat
    
    
    
    feature_mat = np.swapaxes(feature_mat,1,2)
    feature_mat = np.reshape(feature_mat,(feature_mat.shape[0],-1,feature_mat.shape[-1]))
    feature_mat = feature_mat[:,selected,:]
    
    return feature_mat

def bandpass_filter(signal_in,f_band_nom):
    '''
    Filter N channels with cheby type 2 filter of order 4

    Keyword arguments:
    signal_in  -- numpy array of size [NO_channels, NO_samples]
    f_band_nom -- normalized frequency band [freq_start, freq_end]

    Return: filtered signal 
    '''
    NO_channels ,NO_samples = signal_in.shape
    sig_filt = np.zeros((NO_channels ,NO_samples))
    b, a = signal.cheby2(4, 40, f_band_nom, 'bandpass')
    for channel in range(0,NO_channels):
        # used filtfile for preventing phase delay
        sig_filt[channel] = signal.filtfilt(b,a,signal_in[channel,:])

    return sig_filt

def load_bands(Bands,Fs):
    '''
    Normalizng the Bandwidth
    Keyword arguments:
    bandwith -- numpy array containing bandwiths ex. [2,4,8,16,32]
    f_s      -- sampling frequency

    Return: numpy array of normalized frequency bands
    '''
    for i_bands in range(len(Bands)):
        Bands[i_bands] = [ float(Bands[i_bands][0])/(Fs/2) , float(Bands[i_bands][1])/(Fs/2) ]

    return Bands


def hilbert_transform(signal):
    '''
    perform hilbert transform and return envelope of signal
    keyword arguments:
        signal -- signal in shape of (N_channel , Time_samples )
    Return :
        envelope of input signal (N_channel , Time_samples)
    '''
    analytic_signal = hilbert(signal)
    amplitude_envelope = np.abs(analytic_signal)
    return amplitude_envelope
